In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path

artifact_torch_root = Path().absolute().parent.parent

os.chdir(artifact_torch_root)

Path().absolute()

In [ ]:
import pandas as pd
import seaborn as sns
from artifact_core.binary_classification import BinaryFeatureSpec
from artifact_core.libs.resources.categorical.category_store.binary import BinaryCategoryStore
from artifact_experiment import DataSplit
from artifact_experiment.tracking import FilesystemTrackingClient
from matplotlib import pyplot as plt

from demos.binary_classification.config.constants import (
    EXPERIMENT_ID,
    LABEL_FEATURE,
    LS_CATEGORIES,
    LS_FEATURES,
    POSITIVE_CATEGORY,
    TRAINING_DATASET_PATH,
)
from demos.binary_classification.experiment.experiment import DemoBinaryClassificationExperiment
from demos.binary_classification.model.classifier import MLPClassifier

In [ ]:
sns.set_theme(style="whitegrid", palette="colorblind")

In [ ]:
df_train = pd.read_csv(artifact_torch_root / TRAINING_DATASET_PATH)

df_train

In [ ]:
class_spec = BinaryFeatureSpec(
    ls_categories=LS_CATEGORIES, positive_category=POSITIVE_CATEGORY, feature_name=LABEL_FEATURE
)

class_spec

In [ ]:
from artifact_torch.base.data.data_loader import DataLoader

from demos.binary_classification.config.constants import BATCH_SIZE, DROP_LAST, SHUFFLE
from demos.binary_classification.data.dataset import MLPClassifierDataset

data_loaders = {
    DataSplit.TRAIN: DataLoader(
        dataset=MLPClassifierDataset(
            df=df_train, ls_features=LS_FEATURES, label_feature=class_spec.feature_name
        ),
        batch_size=BATCH_SIZE,
        drop_last=DROP_LAST,
        shuffle=SHUFFLE,
    )
}

In [ ]:
from artifact_torch.binary_classification.routine import BinaryClassificationRoutineData

classification_data = df_train.drop(class_spec.feature_name, axis=1)

id_to_category = df_train[class_spec.feature_name].to_dict()
true_category_store = BinaryCategoryStore.from_categories_and_spec(
    feature_spec=class_spec, id_to_category=id_to_category
)

artifact_routine_data = {
    DataSplit.TRAIN: BinaryClassificationRoutineData[pd.DataFrame](
        true_category_store=true_category_store, classification_data=classification_data
    )
}

In [ ]:
model = MLPClassifier.build(class_spec=class_spec, ls_features=LS_FEATURES)

In [ ]:
tracking_client = FilesystemTrackingClient.build(experiment_id=EXPERIMENT_ID)

In [ ]:
experiment = DemoBinaryClassificationExperiment.build(
    model=model,
    data_loaders=data_loaders,
    artifact_routine_data=artifact_routine_data,
    artifact_routine_data_spec=class_spec,
    tracking_client=tracking_client,
)

In [ ]:
experiment.run()

In [ ]:
plt.figure(figsize=(10, 6))
experiment.epoch_scores["LOSS_TRAIN"].plot(color=sns.color_palette("colorblind")[2], linewidth=2)
plt.title("Train Loss", fontsize=16, fontweight="bold")
plt.xlabel("Epoch", fontsize=14)
plt.ylabel("Training Loss", fontsize=14)
plt.grid(True, linestyle="--", alpha=0.6)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()